In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

# Read the CSV and Perform Basic Data Cleaning

In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
# See the columns
list(df.columns)

['koi_disposition',
 'koi_fpflag_nt',
 'koi_fpflag_ss',
 'koi_fpflag_co',
 'koi_fpflag_ec',
 'koi_period',
 'koi_period_err1',
 'koi_period_err2',
 'koi_time0bk',
 'koi_time0bk_err1',
 'koi_time0bk_err2',
 'koi_impact',
 'koi_impact_err1',
 'koi_impact_err2',
 'koi_duration',
 'koi_duration_err1',
 'koi_duration_err2',
 'koi_depth',
 'koi_depth_err1',
 'koi_depth_err2',
 'koi_prad',
 'koi_prad_err1',
 'koi_prad_err2',
 'koi_teq',
 'koi_insol',
 'koi_insol_err1',
 'koi_insol_err2',
 'koi_model_snr',
 'koi_tce_plnt_num',
 'koi_steff',
 'koi_steff_err1',
 'koi_steff_err2',
 'koi_slogg',
 'koi_slogg_err1',
 'koi_slogg_err2',
 'koi_srad',
 'koi_srad_err1',
 'koi_srad_err2',
 'ra',
 'dec',
 'koi_kepmag']

# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_period', 
                        'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad',
                       'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_steff', 'koi_slogg',
                       'koi_srad', 'ra', 'dec', 'koi_kepmag']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
# Set y value
y = df['koi_disposition'].values.reshape(-1, 1)
y

array([['CONFIRMED'],
       ['FALSE POSITIVE'],
       ['FALSE POSITIVE'],
       ...,
       ['CANDIDATE'],
       ['FALSE POSITIVE'],
       ['FALSE POSITIVE']], dtype=object)

In [6]:
# Apply encoding for y
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

/Users/yyh/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [7]:
# # Apply one-hot encoding for y
# from keras.utils import to_categorical

# # Step 2: One-hot encoding
# one_hot_y = to_categorical(encoded_y)
# one_hot_y

Using TensorFlow backend.


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, encoded_y, random_state=42)
# SVM uses 1 dimension

In [23]:
# Display sample from X_train
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6122,0,0,0,6.768901,133.077240,0.150,3.61600,123.1,1.24,1017,253.30,10.8,5737,4.327,1.125,294.40472,39.351681,14.725
6370,0,1,0,0.733726,132.020050,0.291,2.30900,114.6,0.86,1867,2891.64,13.8,5855,4.578,0.797,284.50391,42.463860,15.770
2879,1,0,0,7.652707,134.460380,0.970,79.89690,641.1,3.21,989,226.81,254.3,6328,4.481,0.963,295.50211,38.983540,13.099
107,0,0,0,7.953547,174.662240,0.300,2.63120,875.4,2.25,696,55.37,38.4,4768,4.536,0.779,291.15878,40.750271,15.660
29,0,0,0,4.959319,172.258529,0.831,2.22739,9802.0,12.21,1103,349.40,696.5,5712,4.359,1.082,292.16705,48.727589,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [24]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaler = scaler.fit(X_train)

In [25]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [26]:
# Create a support vector machine linear classifer and fit it to the training data
from sklearn.svm import SVC 
model = SVC(kernel='rbf')
model.fit(X_train_scaled, y_train)

SVC()

In [29]:
# Print the model score using the test data
predictions = model.predict(X_test_scaled)
print("Test Acc:", model.score(X_test_scaled, y_test))

Test Acc: 0.7820366132723112


In [30]:
# Calculate the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["CANDIDATE", "CONFIRMED", "FALSE POSITIVE"]))

                precision    recall  f1-score   support

     CANDIDATE       0.59      0.43      0.50       411
     CONFIRMED       0.60      0.75      0.67       484
FALSE POSITIVE       0.98      0.97      0.97       853

      accuracy                           0.78      1748
     macro avg       0.72      0.72      0.71      1748
  weighted avg       0.78      0.78      0.78      1748



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [31]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}

grid = GridSearchCV(model, param_grid, verbose=3)

In [32]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ..............................C=1, gamma=0.0001; total time=   2.2s
[CV 2/5] END ..............................C=1, gamma=0.0001; total time=   2.0s
[CV 3/5] END ..............................C=1, gamma=0.0001; total time=   2.0s
[CV 4/5] END ..............................C=1, gamma=0.0001; total time=   2.0s
[CV 5/5] END ..............................C=1, gamma=0.0001; total time=   2.0s
[CV 1/5] END ...............................C=1, gamma=0.001; total time=   2.1s
[CV 2/5] END ...............................C=1, gamma=0.001; total time=   2.3s
[CV 3/5] END ...............................C=1, gamma=0.001; total time=   2.0s
[CV 4/5] END ...............................C=1, gamma=0.001; total time=   2.0s
[CV 5/5] END ...............................C=1, gamma=0.001; total time=   2.0s
[CV 1/5] END ................................C=1, gamma=0.01; total time=   1.3s
[CV 2/5] END ................................C=1,

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             verbose=3)

In [33]:
# List the best parameters for this dataset
# List the best score
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'gamma': 0.01}
0.7823759814872762


In [34]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [35]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["CANDIDATE", "CONFIRMED", "FALSE POSITIVE"]))

                precision    recall  f1-score   support

     CANDIDATE       0.62      0.39      0.48       411
     CONFIRMED       0.60      0.79      0.68       484
FALSE POSITIVE       0.98      0.97      0.97       853

      accuracy                           0.78      1748
     macro avg       0.73      0.72      0.71      1748
  weighted avg       0.79      0.78      0.78      1748



# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# import joblib
# filename = 'your_name.sav'
# joblib.dump(your_model, filename)